# Scraping of EU SUSTAINABLE DEVELOPMENT GOALS datasets (https://ec.europa.eu/eurostat/web/sdi/database)

### 1- Scraping of datasetCodes

In [15]:
import requests
import time
import re
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

import eurostat
from sqlalchemy import create_engine

In [2]:
# Attention à la version de Selenium ! : > 4.1 (https://medium.com/@titus_on_testing/shadow-dom-in-python-selenium-df8d731b5005)

url = "https://ec.europa.eu/eurostat/web/sdi/database"

service = Service(executable_path=r'/usr/bin/chromedriver') # A du être modifié en passant à Selenium 4.10.0

options = webdriver.ChromeOptions()
options.add_argument('--headless')

driver = webdriver.Chrome(service = service, options = options)
driver.get(url)

time.sleep(3) #Peut être mieux fait avec les fonctions Wait de selenium

shadow_host = driver.find_element(By.CSS_SELECTOR , 'navigation-tree')
script = 'return arguments[0].shadowRoot.children'
children = driver.execute_script(script, shadow_host)

navigation_tree_htlm = children[0].get_attribute('innerHTML') #On a récupérer le HTML complet du shadow element, on va directement chercher ce qui nous interesse
# Note : On peut sauvegarder l'élement précédent dans un fichier HTML pour voir ce qui est récupérer et par exemple essayer des selecteurs css

driver.quit()

In [3]:
html_parsing = re.findall("sdg_[0-9][0-9]_[0-9][0-9][a-z]?",navigation_tree_htlm)

In [8]:
#On supprimer les doublons, on ne peut pas passer par un set car sinon on désorganise les valeurs

dataset_codes = []
for i in html_parsing :
    if i not in dataset_codes:
        dataset_codes.append(i)

In [11]:
with open("dataset_codes.txt", "w") as output:
    output.write(str(dataset_codes))

### 2- Getting dataset informations

In [16]:
#Database connection
pw = "password1999"
connection_string = 'mysql+pymysql://root:' + pw + '@localhost:3306/'
engine = create_engine(connection_string, pool_pre_ping = False)

In [20]:
df_sdg_01_10 = eurostat.get_data_df("sdg_01_10", flags=True)
df_sdg_01_10.to_sql("sdg_01_10", connection_string, schema="eurostat", if_exists='replace', index = False)

168

In [21]:
for dataset_code in dataset_codes :
    df = eurostat.get_data_df(dataset_code)
    df.to_sql(dataset_code, connection_string, schema="eurostat", if_exists='replace', index = False)